In [2]:
# ==============================================================================
# Step 1: Initialize Session & Analyze Metadata (Path Fixed)
# ==============================================================================
import sys
import os
from pathlib import Path

# 🔧 自动修复路径：尝试将项目根目录添加到系统路径
current_path = Path.cwd()
project_root = None

# 向上查找包含 src 的目录
if (current_path / "src").exists():
    project_root = current_path
elif (current_path.parent / "src").exists():
    project_root = current_path.parent
    sys.path.append(str(project_root))
    print(f"✅ 已将项目根目录添加到路径: {project_root}")

# 再次检查
try:
    from src.session import DataSession
    from src.utils import enable_polars_markdown_display
    import polars as pl
    print("✅ 成功导入 src 模块")
except ImportError:
    print("❌ 仍然无法找到 'src' 模块。请确保：")
    print("1. 你已创建了 'src' 文件夹")
    print("2. 'src' 文件夹内包含 '__init__.py', 'session.py', 'utils.py' 等文件")
    # 如果是在完全没有文件的环境中，这里会停止
    raise

# ------------------------------------------------------------------------------
# 以下是分析逻辑
# ------------------------------------------------------------------------------

# 1. 启用 Markdown 表格输出规范
enable_polars_markdown_display()

# 2. 加载数据会话
dataset_id = "alldata" 
session = DataSession()

# 检查数据是否已存在，不存在则加载
# ⚠️ 如果你没有 alldata 数据集，请确保在这里加载正确的文件，例如 session.load("data.csv")
if f"df_{dataset_id}" not in locals():
    # 为了演示，如果你没有真实数据，这里可能会报错。
    # 如果已有 df_df 变量，可以直接注释掉下面这行
    try:
        session.load(dataset_id, alias="df")
    except Exception as e:
        print(f"ℹ️ 提示: 加载默认数据失败 ({e})") 
        print("   如果你已经手动加载了 dataframe (例如 df = pl.read_csv...)，请忽略此提示。")
else:
    print(f"✅ 数据 {dataset_id} 已在会话中")

# 获取全局数据对象 (尝试获取 df_df，如果不存在则使用 session 中的第一个)
try:
    df_target = df_df
except NameError:
    if session.list_loaded():
        first_key = session.list_loaded()[0]
        df_target = session.get(first_key)
        print(f"⚠️ 使用已加载的第一个数据集: {first_key}")
    else:
        print("❌ 错误: 会话中没有数据。请先加载数据 (例如: session.load('your_file.csv'))")
        raise ValueError("No data loaded")

# 3. 自动化维度/度量识别逻辑
print("\n## 🔍 数据结构自动化分析报告\n")

suggested_dimensions = []
suggested_metrics = []

for col in df_target.columns:
    # 获取元数据
    dtype = str(df_target[col].dtype)
    n_unique = df_target[col].n_unique()
    
    # A. 识别潜在维度 (String 或 Date)
    if dtype == 'String' or dtype.startswith('Date') or dtype == 'Boolean':
        dim_info = {
            'name': col,
            'type': dtype,
            'unique_values': n_unique,
            'recommendation': ''
        }
        
        if n_unique <= 10:
            dim_info['recommendation'] = "Dropdown (单选/全选)"
        elif n_unique <= 50:
            dim_info['recommendation'] = "MultiChoice (多选)"
        elif n_unique <= 500:
            dim_info['recommendation'] = "MultiChoice (带搜索)"
        else:
            dim_info['recommendation'] = "⚠️ 建议作为级联子级或不作为主维度"
            
        suggested_dimensions.append(dim_info)
        
    # B. 识别潜在度量 (Numeric)
    elif dtype in ['Float64', 'Int64', 'Int32', 'Float32']:
        suggested_metrics.append(f"- **{col}** ({dtype})")

# 4. 输出建议供用户确认
print("### ✅ 建议的分析维度 (Dimensions):\n")
print("| 字段名 | 类型 | 唯一值数量 | 建议控件 |")
print("|---|---|---|---|")
for dim in suggested_dimensions:
    print(f"| **{dim['name']}** | {dim['type']} | {dim['unique_values']} | {dim['recommendation']} |")

print("\n### 🔢 建议的度量指标 (Metrics):\n")
for met in suggested_metrics:
    print(met)

print("\n" + "="*80)
print("💡 下一步：请查看上表，并回复确认使用的维度列表 ANALYSIS_DIMENSIONS。")
print("="*80)

❌ 仍然无法找到 'src' 模块。请确保：
1. 你已创建了 'src' 文件夹
2. 'src' 文件夹内包含 '__init__.py', 'session.py', 'utils.py' 等文件


ModuleNotFoundError: No module named 'src'